In [ ]:
# 데이터 구조
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# Gradient Boosting
from xgboost import XGBClassifier
# xgboost
from sklearn.ensemble import GradientBoostingClassifier
# 모델 성능 평가 : F1 Score, ROC Curve, AUC
from sklearn.metrics import f1_score, roc_curve, roc_auc_score

In [ ]:
df_raw = pd.read_csv("../data/bank.csv")
df_raw.head()

In [ ]:
# 결측치 확인
df_raw.isnull().sum()
# JOB 변수의 결측치는 "Other"로 대체
df_raw["JOB"].fillna("Other", inplace = True)
# 숫자형 변수의 결측치는 해당 변수의 평균값으로 대체
df_raw.fillna(df_raw.mean(), inplace=True)

In [ ]:
# get_dummies : 데이터의 문자형 변수에 대한 더미변수 생성
df_raw_dummy = pd.get_dummies(df_raw)
# 더미변수 생성된 데이터의 상위 5개 row를 확인
df_raw_dummy.head()

In [ ]:
# 설명변수, 목표변수 분리
df_raw_x = df_raw_dummy.drop("BAD", axis = 1, inplace = False)
df_raw_y = df_raw_dummy["BAD"]

In [ ]:
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x, df_raw_y, test_size = 0.3, random_state = 1234)
print("학습용 데이터의 X 크기 : {}".format(df_train_x.shape))
print("학습용 데이터의 Y 크기 : {}".format(df_train_y.shape))
print("평가용 데이터의 X 크기 : {}".format(df_test_x.shape))
print("평가용 데이터의 Y 크기 : {}".format(df_test_y.shape))

In [ ]:
# 분석 모델
model = ["DecisionTree", "RandomForest", "GradientBoosting", "XGBoost"]
# 정확도 저장
train_accuracy = []; test_accuracy = []
# f1 score 저장
model_f1_score = []
# auc score 저장
model_auc = []

In [ ]:
# 모델 생성
dt_model = DecisionTreeClassifier(max_depth = 4, min_samples_leaf = 30)
dt_model.fit(df_train_x, df_train_y)
# train 정확도
train_accuracy.append(dt_model.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(dt_model.score(df_test_x, df_test_y))
# 예측값
y_pred = dt_model.predict(df_test_x)
# roc_auc 계산
roc_auc = roc_auc_score(df_test_y, y_pred)
# auc 저장
model_auc.append(roc_auc)
# f1 스코어
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
# 모델 생성
rf_model = RandomForestClassifier(min_samples_leaf = 1, max_depth = 8, n_estimators = 100,
random_state=1234)
rf_model.fit(df_train_x, df_train_y)
# train 정확도
train_accuracy.append(rf_model.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(rf_model.score(df_test_x, df_test_y))
# 예측값
y_pred = rf_model.predict(df_test_x)
# roc_auc 계산
roc_auc = roc_auc_score(df_test_y, y_pred)
# auc 저장
model_auc.append(roc_auc)
# f1 스코어
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
# 모델 생성
gb_final = GradientBoostingClassifier(min_samples_leaf = 5, max_depth = 6, n_estimators = 200,
learning_rate = 0.3, random_state=1234)
gb_final.fit(df_train_x, df_train_y)
# train 정확도
train_accuracy.append(gb_final.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(gb_final.score(df_test_x, df_test_y))
# 예측값
y_pred = gb_final.predict(df_test_x)
# roc_auc 계산
roc_auc = roc_auc_score(df_test_y, y_pred)
# auc 저장
model_auc.append(roc_auc)
# f1 스코어
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
# 모델 생성
xgb_final = XGBClassifier(learning_rate = 0.3, booster = 'gbtree', max_depth = 5,
n_estimators = 150, gamma = 0, objective = 'binary:logistic', random_state=1234)
xgb_final.fit(df_train_x, df_train_y)
# train 정확도
train_accuracy.append(xgb_final.score(df_train_x, df_train_y))
# test 정확도
test_accuracy.append(xgb_final.score(df_test_x, df_test_y))
# 예측값
y_pred = xgb_final.predict(df_test_x)
# roc_auc 계산
roc_auc = roc_auc_score(df_test_y, y_pred)
# auc 저장
model_auc.append(roc_auc)
# f1 스코어
model_f1_score.append(f1_score(df_test_y, y_pred))

In [ ]:
# 모델별 정확도 표
df_eval = pd.DataFrame(index = model)
df_eval["TrainAccuracy"] = train_accuracy
df_eval["TestAccuracy"] = test_accuracy
df_eval["F1Score"] = model_f1_score
df_eval["AUC"] = model_auc
df_eval.round(3)

In [ ]:
# 정확도 bar plot 생성
df_eval.plot.bar(rot = 0)
plt.legend(loc="lower right")
plt.grid(axis = "y")